# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils
from decision_making_analysis.event_detection import detect_rsw_and_rcap

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_psth import core_stops_psth, psth_postprocessing, psth_stats, compare_events, dpca_utils
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import get_stops_utils, collect_stop_data

from neural_data_analysis.neural_analysis_tools.glm_tools.glm_fit import general_glm_fit, cv_stop_glm, glm_fit_utils, variance_explained, glm_runner
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_plotting import plot_spikes, plot_glm_fit, plot_tuning_func
from neural_data_analysis.design_kits.design_around_event import event_binning, stop_design, cluster_design, design_checks
from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.stop_glm.glm_hyperparams import compare_glm_configs, glm_hyperparams_class
from neural_data_analysis.topic_based_neural_analysis.ff_visibility import ff_vis_epochs, vis_design

from neural_data_analysis.topic_based_neural_analysis.stop_event_analysis.get_stop_events import assemble_stop_design

# import decoding
from neural_data_analysis.neural_analysis_tools.decoding_tools.event_decoding import decoding_utils, decoding_analysis, plot_decoding, cmp_decode, load_results

import sys
import math
import gc
import subprocess
from pathlib import Path

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
import cProfile
import pstats
import json

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr
import statsmodels.api as sm

# Neuroscience specific imports
import neo
import rcca

# To fit gpfa
import numpy as np
from importlib import reload
from scipy.integrate import odeint
import quantities as pq
import neo
from elephant.spike_train_generation import inhomogeneous_poisson_process
from elephant.gpfa import GPFA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from elephant.gpfa import gpfa_core, gpfa_util

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

print("done")


%load_ext autoreload
%autoreload 2
%matplotlib inline

pd.set_option('display.max_colwidth', 200)

# retrieve data

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0301"

pn, datasets, comparisons = collect_stop_data.collect_stop_data_func(
    raw_data_folder_path)

globals().update(datasets)

captures_df, valid_captures_df, filtered_no_capture_stops_df, stops_with_stats = get_stops_utils.prepare_no_capture_and_captures(
    monkey_information=pn.monkey_information,
    closest_stop_to_capture_df=pn.closest_stop_to_capture_df,
    ff_caught_T_new=pn.ff_caught_T_new,
    distance_col="distance_from_ff_to_stop",
)

keys = ['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle', 'one_stop_vs_both_first_miss', 'will_switch_vs_retry_after_miss', 'switch_vs_retry_after_retry']

cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.04,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)

## save comparisons.json

In [ ]:
stop!

In [ ]:
# if save all combos

import json, os
os.makedirs("configs", exist_ok=True)
with open("multiff_analysis/configs/comparisons.json", "w") as f:
    json.dump(comparisons, f, indent=2)

In [ ]:
# if only save a subset of comparisons

keys = [
    'rsw_first_vs_rcap_first',
    'rsw_middle_vs_rcap_middle',
    'one_stop_vs_both_first_miss',
    'will_switch_vs_retry_after_miss',
    'switch_vs_retry_after_retry'
]

# Filter the comparisons list by key
comparisons_sub = [c for c in comparisons if c['key'] in keys]


import json, os
os.makedirs("configs", exist_ok=True)
with open("multiff_analysis/configs/comparisons.json", "w") as f:
    json.dump(comparisons_sub, f, indent=2)

In [ ]:
with open("multiff_analysis/configs/comparisons.json", "r") as f:
    comparisons_sub = json.load(f)

In [ ]:
len(comparisons_sub)

# Decoding results

## plot by key (all sessions)

### non-cumulative

In [ ]:
df_all_combined = load_results.load_all_retry_decoder_results(
    monkey_name='monkey_Bruno', cumulative=False)

In [ ]:
df_all_combined_sub = df_all_combined[df_all_combined['align_by_stop_end']==True]
plot_decoding.plot_decoding_timecourse_by_session(
    df_all_combined_sub,
    align_col="align_by_stop_end",
    value_col="mean_auc",
    sig_col="sig_ttest" if "sig_ttest" in df_all_combined.columns else "sig_FDR",
    err_col="sd_auc" if "sd_auc" in df_all_combined.columns else None,
)

### cumulative

In [ ]:
df_all_combined = load_results.load_all_retry_decoder_results(
    monkey_name='monkey_Bruno', cumulative=True)

In [ ]:
df_all_combined_sub = df_all_combined[df_all_combined['align_by_stop_end']==True]
plot_decoding.plot_decoding_timecourse_by_session(
    df_all_combined_sub,
    align_col="align_by_stop_end",
    value_col="mean_auc",
    sig_col="sig_ttest" if "sig_ttest" in df_all_combined.columns else "sig_FDR",
    err_col="sd_auc" if "sd_auc" in df_all_combined.columns else None,
)

In [ ]:
# df_all_combined_sub[(df_all_combined_sub['key']=='guat_first_vs_taft_first')&
#                     (df_all_combined_sub['model_name']=='logreg')&
#                     (df_all_combined_sub['session']=='data_0220')]

## plot by session

In [ ]:
from data_wrangling import combine_info_utils, specific_utils
# df_all_combined = pd.DataFrame()

monkey_name = 'monkey_Bruno'
num_sessions = 5

raw_data_dir_name = 'all_monkey_data/raw_monkey_data'
sessions_df_for_one_monkey = combine_info_utils.make_sessions_df_for_one_monkey(
                raw_data_dir_name, monkey_name)


session_count = 0
for index, row in sessions_df_for_one_monkey.iterrows():
    print(row['data_name'])
    raw_data_folder_path = os.path.join(
        raw_data_dir_name, row['monkey_name'], row['data_name'])
    print(raw_data_folder_path)
    
    try:
        df_all = cmp_decode.summarize_and_plot_decoding(raw_data_folder_path, cumulative=True)   
    except Exception as e:
        print(f"Error in {raw_data_folder_path}: {e}")
        continue
    
    df_all['session'] = row['data_name']
    sessions_df_for_one_monkey.loc[index, 'finished'] = True
    # df_all_combined = pd.concat([df_all_combined, df_all], ignore_index=True)
    
    session_count += 1
    # if session_count >= num_sessions:
    #     break

In [ ]:
stop!

## plot one session

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0301"
df_all = cmp_decode.summarize_and_plot_decoding(raw_data_folder_path, cumulative=False)   

In [ ]:
# check the uniqueness of entries in df_all
df_all.groupby(['key', 'window_start', 'model_name', 'align_by_stop_end']).count().max()

In [ ]:
summary = _summarize(df_all)
cmp_decode._plot_summary_bars(summary, title_prefix="Peak AUC by model")

# How much do models agree

## all models

### combine all keys

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = df_all_combined.copy()
key_cols = ['window_start', 'window_end', 'key', 'session']

# ----------------------------------------------------------
# 1️⃣ Compute Jaccard overlap (significant only)
df_sig = df[df['sig_ttest']]

sig_sets = {
    model: set(map(tuple, g[key_cols].values))
    for model, g in df_sig.groupby('model_name')
}
models = list(sig_sets.keys())

jaccard_sig = np.zeros((len(models), len(models)))
for i, m1 in enumerate(models):
    for j, m2 in enumerate(models):
        inter = len(sig_sets[m1] & sig_sets[m2])
        union = len(sig_sets[m1] | sig_sets[m2])
        jaccard_sig[i, j] = inter / union if union > 0 else np.nan
jaccard_sig_df = pd.DataFrame(jaccard_sig, index=models, columns=models)

# ----------------------------------------------------------
# 2️⃣ Compute Jaccard overlap (all windows)
all_sets = {
    model: set(map(tuple, g[key_cols].values))
    for model, g in df.groupby('model_name')
}
jaccard_all = np.zeros((len(models), len(models)))
for i, m1 in enumerate(models):
    for j, m2 in enumerate(models):
        inter = len(all_sets[m1] & all_sets[m2])
        union = len(all_sets[m1] | all_sets[m2])
        jaccard_all[i, j] = inter / union if union > 0 else np.nan
jaccard_all_df = pd.DataFrame(jaccard_all, index=models, columns=models)

# ----------------------------------------------------------
# 3️⃣ Compute AUC correlation (significant only)
sig_auc = df_sig.pivot_table(index=key_cols, columns='model_name', values='mean_auc')
auc_corr_sig = sig_auc.corr(method='pearson')

# 4️⃣ Compute AUC correlation (all windows)
auc_all = df.pivot_table(index=key_cols, columns='model_name', values='mean_auc')
auc_corr_all = auc_all.corr(method='pearson')

# ----------------------------------------------------------
# 🖼 Plot: 2×2 grid of heatmaps
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sns.heatmap(
    jaccard_sig_df, ax=axes[0, 0], cmap='YlGnBu', vmin=0, vmax=1, annot=True, fmt='.2f',
    cbar_kws={'label': 'Jaccard overlap'}
)
axes[0, 0].set_title('Significant windows only — Jaccard overlap')

sns.heatmap(
    auc_corr_sig, ax=axes[0, 1], cmap='viridis', vmin=0, vmax=1, annot=True, fmt='.2f',
    cbar_kws={'label': 'Pearson r'}
)
axes[0, 1].set_title('Significant windows only — AUC correlation')

sns.heatmap(
    jaccard_all_df, ax=axes[1, 0], cmap='YlGnBu', vmin=0, vmax=1, annot=True, fmt='.2f',
    cbar_kws={'label': 'Jaccard overlap'}
)
axes[1, 0].set_title('All windows — Jaccard overlap')

sns.heatmap(
    auc_corr_all, ax=axes[1, 1], cmap='viridis', vmin=0, vmax=1, annot=True, fmt='.2f',
    cbar_kws={'label': 'Pearson r'}
)
axes[1, 1].set_title('All windows — AUC correlation')

for ax in axes.flat:
    ax.set_xlabel('Model')
    ax.set_ylabel('Model')

plt.suptitle('Model similarity across significance scope', fontsize=14, y=1.03)
plt.tight_layout()
plt.show()

### for each key

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df = df_all_combined.copy()

# Keep only significant windows
df_sig = df[df['sig_ttest']]

# Define key columns
key_cols = ['window_start', 'window_end', 'key', 'session']

# Prepare output containers
all_jaccards = {}
all_auc_corrs = {}

# Loop over decoding contrasts
for key, df_key in df_sig.groupby('key'):
    # --- Jaccard overlaps ---
    sig_sets = {
        model: set(map(tuple, g[key_cols].values))
        for model, g in df_key.groupby('model_name')
    }
    models = list(sig_sets.keys())
    jaccard = np.zeros((len(models), len(models)))
    for i, m1 in enumerate(models):
        for j, m2 in enumerate(models):
            inter = len(sig_sets[m1] & sig_sets[m2])
            union = len(sig_sets[m1] | sig_sets[m2])
            jaccard[i, j] = inter / union if union > 0 else np.nan

    jaccard_df = pd.DataFrame(jaccard, index=models, columns=models)
    all_jaccards[key] = jaccard_df

    # --- AUC correlations ---
    sig_auc = df_key.pivot_table(index=key_cols, columns='model_name', values='mean_auc')
    auc_corr = sig_auc.corr(method='pearson')
    all_auc_corrs[key] = auc_corr

    # --- Plot heatmaps ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    sns.heatmap(jaccard_df, ax=axes[0], annot=True, cmap='YlGnBu', vmin=0, vmax=1)
    axes[0].set_title(f'{key}\nJaccard overlap of significant windows')
    sns.heatmap(auc_corr, ax=axes[1], annot=True, cmap='viridis', vmin=0, vmax=1)
    axes[1].set_title(f'{key}\nAUC correlation (significant windows)')
    plt.tight_layout()
    plt.show()

## between 2 models

### both windows and auc

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import textwrap

# --- Setup ---
df = df_all_combined.copy()
key_cols = ['window_start', 'window_end', 'key', 'session']
model_a, model_b = 'svm', 'logreg'

# --- Helper ---
def compute_metrics(df_subset):
    results = []
    for key, df_key in df_subset.groupby('key'):
        sets = {m: set(map(tuple, g[key_cols].values)) for m, g in df_key.groupby('model_name')}
        if model_a not in sets or model_b not in sets:
            continue

        inter = len(sets[model_a] & sets[model_b])
        union = len(sets[model_a] | sets[model_b])
        jaccard = inter / union if union > 0 else np.nan

        sig_auc = df_key.pivot_table(index=key_cols, columns='model_name', values='mean_auc')
        if model_a in sig_auc and model_b in sig_auc:
            valid = sig_auc[[model_a, model_b]].dropna()
            auc_corr = valid[model_a].corr(valid[model_b]) if not valid.empty else np.nan
        else:
            auc_corr = np.nan

        results.append({'key': key, 'jaccard': jaccard, 'auc_corr': auc_corr})
    return pd.DataFrame(results)

# --- Compute for sig-only and all ---
df_sig = df[df['sig_ttest']]
res_sig = compute_metrics(df_sig)
res_all = compute_metrics(df)

df_res = res_sig.merge(res_all, on='key', suffixes=('_sig', '_all')).sort_values('jaccard_sig', ascending=False)

# --- Shorten and wrap key labels for readability ---
def clean_label(k):
    # simplify and shorten text, but keep it all on one line
    k = (k.replace('captures_vs_', 'cap vs ')
           .replace('first_', '1st ')
           .replace('_vs_', ' vs ')
           .replace('_', ' '))
    return k.strip()


df_res['label'] = df_res['key'].apply(clean_label)

# --- Prepare tidy format for plotting ---
plot_df = pd.melt(
    df_res,
    id_vars='label',
    value_vars=['jaccard_sig', 'auc_corr_sig', 'auc_corr_all'],
    var_name='metric',
    value_name='value'
)

metric_labels = {
    'jaccard_sig': 'Jaccard (sig-only)',
    'auc_corr_sig': 'AUC corr (sig-only)',
    'auc_corr_all': 'AUC corr (all)'
}
plot_df['metric'] = plot_df['metric'].map(metric_labels)

# --- Plot ---
plt.figure(figsize=(13, 5))
sns.barplot(
    data=plot_df,
    x='label', y='value', hue='metric',
    palette=['C0', 'C2', 'gold'], edgecolor='black'
)

plt.ylim(0, 1)
plt.ylabel('Value')
plt.xlabel('Decoding contrast (key)')
plt.title(f'Model similarity across decoding contrasts\n({model_a} vs {model_b})', pad=12)

# Rotate x labels neatly
plt.xticks(rotation=30, ha='right', fontsize=9)
plt.legend(frameon=False, bbox_to_anchor=(1.02, 1), loc='upper left')
sns.despine()
plt.tight_layout()
plt.show()

### just sig windows

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = df_all_combined.copy()
df_sig = df[df['sig_ttest']]
key_cols = ['window_start', 'window_end', 'key', 'session']

model_a, model_b = 'svm', 'logreg'

# Compute per-key Jaccard overlap
overlap_scores = {}
for key, df_key in df_sig.groupby('key'):
    sets = {
        m: set(map(tuple, g[key_cols].values))
        for m, g in df_key.groupby('model_name')
    }
    if model_a in sets and model_b in sets:
        inter = len(sets[model_a] & sets[model_b])
        union = len(sets[model_a] | sets[model_b])
        jaccard = inter / union if union > 0 else 0
        overlap_scores[key] = jaccard

# Convert to tidy DataFrame
df_overlap = pd.DataFrame(list(overlap_scores.items()), columns=['key', 'jaccard'])
df_overlap = df_overlap.sort_values('jaccard', ascending=False)

# Plot
plt.figure(figsize=(8, 4))
sns.barplot(data=df_overlap, x='key', y='jaccard', palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1)
plt.ylabel('Jaccard overlap (SVM vs LogReg)')
plt.xlabel('Decoding contrast (key)')
plt.title('Model agreement across decoding contrasts')
for i, v in enumerate(df_overlap['jaccard']):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center', va='bottom', fontsize=9)
plt.tight_layout()
plt.show()


# See best params

## All sessions

In [ ]:
if 'df_all_combined' not in globals():
    df_all_combined = load_results.load_all_retry_decoder_results(
        monkey_name='monkey_Bruno', cumulative=True)

In [ ]:
# Single param
cmp_decode.plot_best_params_3d(df_all_combined, 'logreg', param_x='C')

# Two params
cmp_decode.plot_best_params_3d(df_all_combined, 'svm', param_x='C', param_y='gamma')

# Three params
cmp_decode.plot_best_params_3d(df_all_combined, 'rf', param_x='max_depth', param_y='min_samples_leaf', param_z='n_estimators')


cmp_decode.plot_best_params_3d(df_all_combined, 'logreg_elasticnet', param_x='C', param_y='l1_ratio')



In [ ]:
df_all_combined[df_all_combined['model_name'] == 'rf']['best_params']

## One session

In [ ]:
pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(
        raw_data_folder_path=raw_data_folder_path)
job_result_dir = Path(pn.retry_decoder_folder_path) / "runs"
df_all = cmp_decode._load_all_results(job_result_dir, None)
cmp_decode.plot_best_logreg_elasticnet_params(df_all)

# Reset cluster run progress

In [ ]:
# with open(progress_path, "r") as f:
#     progress = json.load(f)
# progress

In [ ]:

import os, json, hashlib
from pathlib import Path
from datetime import datetime

monkey_dir = Path('all_monkey_data/raw_monkey_data/monkey_Bruno')
retry_monkey_dir = Path('all_monkey_data/retry_decoder/monkey_Bruno')
retry_monkey_dir.mkdir(parents=True, exist_ok=True)

progress_filename = f"_decoding_progress_all.json"
progress_path = retry_monkey_dir / progress_filename

all_sessions = []
if monkey_dir.exists():
    all_sessions = sorted([p.name for p in monkey_dir.iterdir() if p.is_dir() and p.name.startswith('data_')])

with open(progress_path, "r") as f:
    progress = json.load(f)
    
models = progress['models']
    
progress = {
    'monkey': monkey_dir.name if monkey_dir else None,
    'all': all_sessions,
    'done': [],
    'pending': all_sessions,
    'last_updated': datetime.now().isoformat(timespec='seconds'),
    'keys': progress['keys'],
    'models': progress['models'],
    'per_model_done': {m: [] for m in models} if models else {},
}

with open(progress_path, 'w') as f:
    json.dump(progress, f, indent=2)

print(f"[SLURM] Progress reset: {progress_path} | done_all=0 | pending_all={len(progress['pending'])}")


# Decode (run locally)

In [ ]:
windows = [(round(t, 1), round(t + 0.1, 1)) for t in np.arange(-0.2, 0.2, 0.1)]
df_results = decoding_analysis.run_all_decoding_comparisons_cumulative(
    comparisons=comparisons, keys=keys, datasets=datasets, pn=pn, cfg=cfg,
    model_name='svm',
    n_perm=200,
    tune=True,
    windows=windows,
    save_dir=Path(pn.retry_decoder_folder_path) / 'cum_window_runs',
    exists_ok=False
)

In [ ]:
keys = ['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle', 'one_stop_vs_both_first_miss', 'will_switch_vs_retry_after_miss', 'switch_vs_retry_after_retry']

In [ ]:
from datetime import datetime
from pathlib import Path

model_name = 'svm'
do_testing = True

timestamp = datetime.now().strftime('%Y%m%d_%H%M')
save_dir = Path(pn.retry_decoder_folder_path) / "runs"

In [ ]:
#windows = [(t, t + 0.1) for t in np.arange(-0.2, 0.3, 0.1)]
# windows = [(round(t, 2), round(t + 0.1, 2)) for t in np.arange(-0.2, 0.3, 0.05)]

windows = [[-0.2, 0.2]]

out_base = Path(pn.retry_decoder_folder_path) / 'temp_runs'

df_results = decoding_analysis.run_all_decoding_comparisons(
    comparisons=comparisons,
    keys=['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle',
          'one_stop_vs_both_first_miss', 'will_switch_vs_retry_after_miss',
          'switch_vs_retry_after_retry'],
    datasets=datasets,
    pn=pn,
    cfg=cfg,
    model_name='logreg_elasticnet', # 'logreg_elasticnet'
    #model_kwargs={'C': 2.0, 'gamma': 0.05},
    tune=False,
    k=5,
    n_perm=5,          # >0 enables permutation testing
    alpha=0.05,
    windows=windows,
    do_testing=True,    # <— set to False to skip all tests,
    save_dir=out_base,
    overwrite=False,
    exists_ok=False
)

In [ ]:
# X, y, unit_ids = decoding_utils.build_Xy(an, window=(0,0.2))
# real_auc, auc_null, p_perm = decoding_utils.permutation_test_auc(X, y)
# mean_auc, sd, tstat, p_ttest, aucs = decoding_utils.ttest_auc_folds(X, y)
# print(f"Permutation p={p_perm:.4f}, t-test p={p_ttest:.4f}")

# Plot heatmap

In [ ]:
df_results['mean_auc'].unique()

In [ ]:
df_results_sub = df_results[df_results['align_by_stop_end']==True].copy()
# df_results_sub = df_results_sub[df_results_sub['key'].isin(keys)]

plot_decoding.plot_decoding_auc_heatmap(
    df_results_sub, threshold=0.55, cmap='magma', title='Align by stop end',
    x_tick_position='center',
    sig_col='p_perm_sig',
)
plt.show()

In [ ]:
# df_results = df_results.sort_values(by=['key', 'a_label', 'b_label', 'window_start', 'align_by_stop_end', 'n_perm']).groupby(['key', 'a_label', 'b_label', 'window_start', 'align_by_stop_end']).last()

In [ ]:
df_results_sub[(df_results_sub['a_label'] == 'rsw first') & (df_results_sub['b_label'] == 'rcap first')]

In [ ]:
df_results_sub = df_results[df_results['align_by_stop_end']==False].copy()
df_results_sub = df_results_sub[df_results_sub['key'].isin(keys)]
plot_decoding.plot_decoding_auc_heatmap(
    df_results_sub, threshold=0.55, cmap='magma', title='Align by stop beginning',
    sig_col='p_perm_sig'
)
plt.show()

In [ ]:
df_results

# Plot

In [ ]:
df_results_sub = df_results[df_results['align_by_stop_end']==True].copy()
df_results_sub = df_results_sub[df_results_sub['key'].isin(keys)]


plot_decoding.plot_decoding_timecourse(
    df_results_sub,
    groupby_cols=('align_by_stop_end',),
    split_by=('a_label','b_label'),
    value_col='mean_auc',
    sig_col='p_perm_sig',
    err_col='sd_auc',
    err_type='sem',
    title_prefix='Decoding timecourse per comparison'
)

In [ ]:
plot_decoding.plot_decoding_timecourse(
    df_results,
    groupby_cols=('a_label','b_label'),
    split_by=('align_by_stop_end'),
    title_prefix='Decoding per alignment'
)

In [ ]:
df_results[(df_results['a_label']=='one stop miss') & (df_results['b_label']=='both first miss')]

In [ ]:
# res = decoding_utils.run_decoding(
#     an,
#     model_name='svm',
#     tune=False,
#     model_kwargs={'C': 2.0, 'gamma': 0.05}
# )

# res = decoding_utils.run_decoding(an, model_name='rf', tune=False, search='grid')

# res = run_decoding(an, model_name='mlp', tune=False, search='random', n_iter=20)

# PSTH

In [ ]:
cfg = core_stops_psth.PSTHConfig(
    pre_window=0.5,
    post_window=0.5,
    bin_width=0.04,
    smoothing_sigma=0.1,
    min_trials=5,
    normalize="zscore",            # try: None, "sub", or "div"
)


keys = ['rsw_first_vs_rcap_first', 'rsw_middle_vs_rcap_middle', 'one_stop_vs_both_first_miss', 'switch_vs_retry_after_retry']
comparisons_sub = [c for c in comparisons if c['key'] in keys]

runs = compare_events.run_all_comparisons(
    comparisons_sub, datasets, pn.spikes_df, pn.monkey_information, cfg,
    align_by_stop_end=True
)